<p style="font-size:260%;line-height:1.5">Insert a compact source from the QUBIC catalog into a Healpix map </p>
<p style="font-size:140%;line-height:1.5">Notebook help file</p>


# Brief intro to the module

<p style="font-size:120%;line-height:1.5">
We use this module when we want to add one or more point sources to a sky generated by PySM. The module uses the QUBIC compact source catalog pickle file that contains a reduced version of the Planck Catalog of Compact Sources n. 2 (PCCS2).
</p>

<p style="font-size:120%;line-height:1.5">
We use the position and flux of the source and smooth it with a gaussian with a user-defined fwhm converting it to K_CMB. The gaussian is defined as follows. It has maximum at 1 and its integral is $2\pi\sigma^2$
</p>

<p style="font-size:120%;line-height:1.5">
\begin{equation}
P_n(\theta,\phi) = e^{-\frac{(\theta-\theta_0)^2 + (\phi-\phi_0)^2}{2\sigma^2}}
\end{equation}
</p>

<p style="font-size:120%;line-height:1.5">
The flux density in Jy/sr at any point $(\theta, \phi)$ is given by
$$
I_\nu = \frac{F(\theta_0,\phi_0)P_n(\theta,\phi)}{\int_{4\pi}P_n(\theta,\phi)d\Omega}
$$
</p> 

<p style="font-size:120%;line-height:1.5">
The conversion between flux density in Jy/sr to K_CMB is provided by the astropy library and it is equal to:
$$
T[\,\mathrm{K_{CMB}}] = \frac{c^2}{2\,k\,\nu^2}\frac{(e^x-1)^2}{x^2e^x}\times I_\nu\,[\mathrm{Jy/sr}]
$$
</p>

<p style="font-size:120%;line-height:1.5">
 For polarization the $Q$ and $U$ components are calculated according to the following equation:
</p>

<p style="font-size:120%;line-height:1.5">
$$
Q = P\cos{(2\Psi)},\,U = P\sin{(2\Psi)}
$$
</p> 

<p style="font-size:120%;line-height:1.5">
\noindent where $\Psi$ is the polarization angle and $P$ the point source polarized flux. Both are derived from the PCCS. The total intensity flux in mJy is extracted from the catalog field specified by the 'DETFLUX' keyword, the polarized flux in mJy is extracted from the catalog field specified by the 'PFLUX' keyword, the polarization angle in degrees is extracted from the catalog field specified by the 'ANGLE_P' keyword.
</p>

# Tests

## Imports

In [ ]:
import healpy as h
import numpy as np
import pickle
import qubic
import qubic.compact_sources_sed as pccs
import pylab as pl
import qubic.insert_point_sources_in_sky as ipss

## Wrapper to PySM

In [ ]:
def generate_input_sky(\
                       instrument_dictionary,\
                       sky_config = {\
                                      'cmb' : None,\
                                      'freefree' : 'f1',\
                                      'synchrotron': 's1',\
                                      'dust' : 'd1'\
                                     }):
    '''
    Fuction that generates an input sky using PySM

    INPUTS
    instrument_dictionary  DICT  the dictionary containing all the instrumental parameters
    sky_config             DICT  the configuration of the sky to be generated. By default it contains CMB,
                                 synchrotron, dust and free free.
    
    OUTPUTS
    sky_map               ARRAY  Shape[instrument_dictionary['nf_sub'], 12*instrument_dictionary['nside']**2,
                                 3], the array of IQU maps at the various sub-frequencies defined in the 
                                 dictionary
    '''

    from qubic import QubicSkySim as qss
    import healpy as hp
    import numpy as np
    
    Qubic_sky = qss.Qubic_sky(sky_config, instrument_dictionary)
    x0 = Qubic_sky.get_simple_sky_map()

    print('Input Map with shape (nf_sub, #pixels, #stokes) : ', np.shape(x0))

    # Check size map
    if hp.get_nside(x0[0,:,0]) == d['nside']:
        print('Good size')
        sky_map = x0
    else:
        print('Bad size')
        sky_map = np.empty((d['nf_sub'], 12 * d['nside'] ** 2, 3))
        for i in range(d['nf_sub']):
            for j in range(3):
                sky_map[i, :, j] = hp.ud_grade(x0[i, :, j], d['nside'])
    
    return sky_map

## Generate map of the Crab at 143 GHz and compare with HFI 143 GHz data

<p style="font-size:120%;line-height:1.5">
Here we want to generate an intensity map of the Crab and compare it to the source as seen by HFI in the 143 GHz channel. We generate the map at an angular resolution comparable to that of Planck 143. We use the function insert_source.
</p>

In [ ]:
help(ipss.insert_source)

In [ ]:
# This is the Crab

source_center_deg = (184.55057376669635,-5.790729915633918)
fwhm_deg = 7/60
flux_Jy = 167.0963125
frequency = 143e9
nside = 2048

In [ ]:
omap = ipss.insert_source(source_center_deg, fwhm_deg, flux_Jy,nside, units='K_CMB', frequency = frequency)

In [ ]:
h.gnomview(omap, rot=source_center_deg,xsize=50)

<p style="font-size:120%;line-height:1.5">
    Let us check the same region as seen by HFI 143 GHz. Notice that here one needs to download the HFI map from the Planck legacy archive
</p>

In [ ]:
# Read HFI map
hfi_mapfile = '/home/daniele/Documents/QUBIC/HFI_SkyMap_143-field-IQU_2048_R2.02_full.fits'
hfi_map = h.read_map(hfi_mapfile)

In [ ]:
# Display the map
h.gnomview(hfi_map, rot=source_center_deg, xsize = 50)

## Add to PySM dust map

<p style="font-size:120%;line-height:1.5">
Here add a single source to an already existing dust map generated with PySM. We use always the function insert_source
</p>

In [ ]:
dustmap = h.read_map('/home/daniele/Documents/QUBIC/pysm-dust-d1-2048.fits')

In [ ]:
# Generate in uK_CMB to be consistent with PySM units

omap1 = ipss.insert_source(source_center_deg, fwhm_deg, flux_Jy,nside, units='uK_CMB', \
                     frequency = frequency, input_map = dustmap)

In [ ]:
h.gnomview(omap1, rot=source_center_deg, xsize=200, norm='log')

## Get a source from catalog

<p style="font-size:120%;line-height:1.5">
This module uses the QUBIC catalog of point sources that is stored in the data directory qubic.data.PATH. Here we show how to access the module and extract information on the sources
</p>

In [ ]:
catalog = qubic.data.PATH + 'qubic_pccs2.pickle'

In [ ]:
with open(catalog, 'rb') as handle:
    catalog = pickle.load(handle)

The catalog is a dictionary organized as follows:
```
Catalog - 
        |-['030']
             |----[Source 1]
             |----[Source 2]
             |----[Source 3]
             |----..........
        | ['044']
             |----[Source 1]
             |----[Source 2]
             |----[Source 3]
             |----..........
        | ['070']
             |----[Source 1]
             |----[Source 2]
             |----[Source 3]
             |----..........
        | ['100']
             |----[Source 1]
             |----[Source 2]
             |----[Source 3]
             |----..........
        | ['143']
             |----[Source 1]
             |----[Source 2]
             |----[Source 3]
             |----..........
        | ['217']
             |----[Source 1]
             |----[Source 2]
             |----[Source 3]
             |----..........
        | ['353']
             |----[Source 1]
             |----[Source 2]
             |----[Source 3]
             |----..........
```        
For each source we have the following keys:
```
RA, DEC, GLON, GLAT, DETFLUX, DETFLUX_ERR, APERFLUX, APERFLUX_ERR, PSFFLUX, PSFFLUX_ERR, GAUFLUX, GAUFLUX_ERR, PFLUX, PFLUX_ERR, ANGLE_P, ANGLE_P_ERR, APER_P, APER_P_ERR, APER_ANGLE_P, APER_ANGLE_P_ERR
```
For more information regarding the meaning of the various keys refer to the Planck explanatory supplement

The function ```getsource``` is a wrapper to the catalog that returns the information of a certain source. The source can be specified using the catalog name, which is in the format ```XXX.Y±WW.Z```, where ```XXX.Y``` is the galactic longitude in degrees and ```WW.Z``` is the galactic latitude in degrees. One can use also the common name (e.g., ```Crab```) if they are defined in ```qubic.compact_sources_sed.altnames```

In [ ]:
crab = ipss.getsource('Crab',100,catalog)

In [ ]:
print(crab)

In [ ]:
print(crab['NAME'])

Given a certain source we can calculate the SED by fitting the catalog data in intensity and polarization with a polynomial. This can be done by the function ```build_sed``` of the module ```qubic.compact_sources_sed```. See below the help of this function

In [ ]:
help(pccs.build_sed)

In [ ]:
sed=pccs.build_sed(\
            pccs.name2cat(\
                'Crab', qubic.compact_sources_sed.altnames),catalog, plot=True)

## Add sources to a PYSM map array (defined in the QUBIC band)

<p style="font-size:120%;line-height:1.5">
    Here we add two sources, Crab and RCW38, to a sky generated using PySM in the 150 GHz band with frequencies defined in the instrument dictionary
</p>

###    Read the dictionary and define the frequency array corresponding to d['nf_sub']

In [ ]:
dictfilename = 'RealisticScanning-BmodesNoDustNoSystPaper0_2020.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)

f0 = d['filter_nu']
delta_f = f0*d['filter_relative_bandwidth']
fmin = f0 - delta_f/2
fmax = f0 + delta_f/2
nfreq = d['nf_sub']
df = delta_f/(nfreq-1)
f_arr = np.array(list(np.arange(fmin,fmax,df)) + [fmax])

In [ ]:
print('Center frequency is %3.2f GHz' % (f0/1e9))
print('fmin is %3.2f GHz' % (fmin/1e9))
print('fmax is %3.2f GHz' % (fmax/1e9))
print('the array is')
print(f_arr)

#print(f_arr/1e9)

###    Generate the sky containing only dust

In [ ]:
mysky = generate_input_sky(d, sky_config = {'dust':'d1'})

###    Let's get the source names for Crab and RCW38 from the catalog

In [ ]:
catalog = qubic.data.PATH + 'qubic_pccs2.pickle'

with open(catalog, 'rb') as handle:
    catalog = pickle.load(handle)
    
nameCrab  = pccs.name2cat('Crab',pccs.altnames)
nameRCW38 = pccs.name2cat('RCW38',pccs.altnames)

sources = [nameCrab, nameRCW38]

###    Now add the two sources to the dust map. First let us look at the help of the function

In [ ]:
help(ipss.add_sources_to_sky_map)

In [ ]:
newmap = ipss.add_sources_to_sky_map(mysky, f_arr, sources, fwhm_deg = ('Auto', 1.5))

### Mapping results

<p style="font-size:120%;line-height:1.5">
Now let us map the region of the two sources, first in the original dust map and then in the new map. We will do this for the first frequency in the array.
</p>

In [ ]:
# Get source centers
Crab_center  = (catalog['143'][nameCrab]['GLON'],catalog['143'][nameCrab]['GLAT'])
RCW38_center = (catalog['143'][nameRCW38]['GLON'],catalog['143'][nameRCW38]['GLAT'])

#### Crab

Original map, all frequencies

In [ ]:
for index in np.arange(len(f_arr)):
    pl.figure(figsize = (8,8))
    titles = ['Crab - Intensity', 'Crab - Q', 'Crab - U']
    for i in range(3):
        h.gnomview(mysky[index,:,i], \
           sub = (1,3,i+1), \
           rot = Crab_center,\
           title = titles[i],\
           cmap = 'jet',\
           norm = 'hist'\
          )

Map with the point source added, all frequencies

In [ ]:
for index in np.arange(len(f_arr)):
    pl.figure(figsize = (8,8))
    titles = ['Crab - Intensity', 'Crab - Q', 'Crab - U']
    for i in range(3):
        h.gnomview(newmap[index,:,i], \
           sub = (1,3,i+1), \
           rot = Crab_center,\
           title = titles[i],\
           cmap = 'jet',\
           norm = 'hist'\
          )

#### RCW38

Original map, all frequencies

In [ ]:
for index in np.arange(len(f_arr)):
    pl.figure(figsize = (8,8))
    titles = ['RCW38 - Intensity', 'RCW38 - Q', 'RCW38 - U']
    for i in range(3):
        h.gnomview(mysky[index,:,i], \
           sub = (1,3,i+1), \
           rot = RCW38_center,\
           title = titles[i],\
           cmap = 'jet',\
           norm = 'hist'\
          )

Map with the point source added, all frequencies

In [ ]:
for index in np.arange(len(f_arr)):
    pl.figure(figsize = (8,8))
    titles = ['RCW38 - Intensity', 'RCW38 - Q', 'RCW38 - U']
    for i in range(3):
        h.gnomview(newmap[index,:,i], \
           sub = (1,3,i+1), \
           rot = RCW38_center,\
           title = titles[i],\
           cmap = 'jet',\
           norm = 'hist'\
          )